In [1]:
import pygmo as pg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'pygmo'

In [4]:
!pip install pygmo --no-binary :all:


  Using cached pygmo-v2.19.0.tar.gz (3.0 MB)


ERROR: pygmo from https://files.pythonhosted.org/packages/e2/12/090ba61479f60d5177a0048736d09dc028b2d65063ed44cb952df506336f/pygmo-v2.19.0.tar.gz does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
# Define the Rastrigin function
def rastrigin(x):
    return 10 * len(x) + sum([xi**2 - 10 * np.cos(2 * np.pi * xi) for xi in x])

# Define the Sphere function
def sphere(x):
    return sum([xi**2 for xi in x])


In [ ]:
# Rastrigin Problem Class
class rastrigin_problem:
    def fitness(self, x):
        return [rastrigin(x)]  # Fitness value for Rastrigin function
    
    def get_bounds(self):
        return ([-5.12] * len(x), [5.12] * len(x))  # Bounds for each variable

# Sphere Problem Class
class sphere_problem:
    def fitness(self, x):
        return [sphere(x)]  # Fitness value for Sphere function
    
    def get_bounds(self):
        return ([-5.12] * len(x), [5.12] * len(x))  # Bounds for each variable


In [ ]:
# Create the problem instances
n_vars = 2  # Number of variables for both functions
rastrigin_pg_problem = pg.problem(rastrigin_problem())  # Rastrigin Problem
sphere_pg_problem = pg.problem(sphere_problem())  # Sphere Problem


In [ ]:
# Create algorithm instances
algo_cmaes = pg.algorithm(pg.cmaes(gen=100))  # CMA-ES with 100 generations
algo_de = pg.algorithm(pg.de(gen=100))  # Differential Evolution with 100 generations
algo_pso = pg.algorithm(pg.pso(gen=100))  # Particle Swarm Optimization with 100 generations

# Create a list of algorithms
algorithms = [algo_cmaes, algo_de, algo_pso]


In [ ]:
# Function to apply an algorithm to a problem and get results
def run_algorithm(problem, algorithm):
    algo = algorithm
    pop = pg.population(problem, 20)  # 20 individuals in population
    algo.evolve(pop)  # Run the algorithm
    return pop.get_f()[0][0], pop.get_x()[0]  # Return fitness and solution

# Run each algorithm on both problems
results = {
    'Algorithm': [],
    'Problem': [],
    'Solution': [],
    'Fitness Value': []
}

# Test each algorithm on both problems
for algo in algorithms:
    for problem, name in zip([rastrigin_pg_problem, sphere_pg_problem], ['Rastrigin', 'Sphere']):
        fitness, solution = run_algorithm(problem, algo)
        results['Algorithm'].append(algo.get_name())
        results['Problem'].append(name)
        results['Solution'].append(solution)
        results['Fitness Value'].append(fitness)

# Convert results to DataFrame for easy viewing
df_results = pd.DataFrame(results)
df_results


In [ ]:
# Visualizing the results with a bar chart for comparison
fig, ax = plt.subplots(figsize=(10, 6))

# Group by Algorithm and Problem to calculate the best fitness value
df_pivot = df_results.pivot_table(values='Fitness Value', index=['Algorithm', 'Problem'], aggfunc='min')
df_pivot.plot(kind='bar', ax=ax, legend=False)

# Set chart labels and title
ax.set_xlabel('Algorithm & Problem')
ax.set_ylabel('Fitness Value')
ax.set_title('Comparison of Optimization Algorithms on Different Problems')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
### Заключение:
В этом ноутбуке мы использовали библиотеку Pygmo для глобальной оптимизации двух тестовых функций: **Rastrigin** и **Sphere**. Мы применили три различных алгоритма:

1. CMA-ES
2. Дифференциальная эволюция (DE)
3. Оптимизация роя частиц (PSO)

Результаты показывают, какие алгоритмы лучше подходят для решения различных типов задач. Например, CMA-ES может быть более эффективным для сложных многомодальных функций, таких как Rastrigin, в то время как PSO может быть хорош для гладких функций, как Sphere.
